# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [71]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [72]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [73]:
import os
from glob import glob
PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [108]:
# Group by 'Ticker' and apply the function to each group
dd_feat = dd_px.groupby('Ticker', group_keys=False).apply(  
    lambda x: x.assign(  
        # Add 'Close_lag_1' as the previous row's 'Close' value
        Close_lag_1 = x['Close'].shift(1),
        # Add 'Close_lag_adj' as the previous row's 'Adj Close' value
        Close_lag_adj = x['Adj Close'].shift(1)  
    )    
).assign(
    # Calculate returns: current Close / previous Close - 1
    returns =  lambda x: x['Close'] / x['Close_lag_1'] - 1,
    # Calculate the range between High and Low values
    hi_lo_range =  lambda x: x['High'] - x['Low']
)

# Return the updated DataFrame
dd_feat

/var/folders/_n/w8k9zwj57xg4g7tchwrj9h1c0000gn/T/ipykernel_58310/1980955981.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('Ticker', group_keys=False).apply(  # group by Ticker value and


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_adj,returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [76]:
# Convert the Dask DataFrame to a Pandas DataFrame
pd_df_feat = dd_feat.compute()

# Make a copy of the DataFrame so we don’t mess with the original
df = pd_df_feat.copy()

# Create a new column 'mv_avg' with a 10-period rolling mean of 'returns'
df['mv_avg'] = df['returns'].rolling(window=10).mean()

# Return the updated DataFrame
df


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    + No, it is possible to calculate moving average in Dask.
+ Would it have been better to do it in Dask? Why?
    + Yes. Doing it in Dask would conserve memory and leverage parallelized computing of the dataframe.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.